In [1]:
"""
Description : Complete model creation process as a single entity for naive bayes 
Grid search has been used
TODO - Test Grid Search better , Choose a better metric?

"""

'\nDescription : Complete model creation process as a single entity for naive bayes \nGrid search has been used\nTODO - Test Grid Search better , Choose a better metric?\n\n'

In [2]:
#importing libraries

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
def navie_bayes_tfidf_model(x,y,itera=1000):
    
    """
    Input  : x=Attributes of the dataset, y=Target Attribute
    Output : Print model details
    
    """
    
    train_x,train_y,test_x,test_y=split_data(x,y)
    
    print("train_x :",train_x.shape )
    print("train_y :",train_y.shape )
    print("test_x :",test_x.shape )
    print("test_y :",test_y.shape )
    
    tfidf_train_x,tfidf_test_x=tfidf_vect(train_x,test_x)

    
    navie_bayes(tfidf_train_x,train_y,tfidf_test_x,test_y,itera=1000)   

In [4]:
def navie_bayes_hash_model(x,y,itera=1000):
    
    """
    Input  : x=Attributes of the dataset, y=Target Attribute
    Output : Print model details
    
    """
    train_x,train_y,test_x,test_y=split_data(x,y)
    
    tfidf_train_x,tfidf_test_x=hash_vect(train_x,test_x)
    
    navie_bayes(tfidf_train_x,train_y,tfidf_test_x,test_y,itera=1000)   

In [5]:
def split_data(x,y):
    
    """
    Reminder : Clean the set before sending here.
    Input  : x=Attributes of the dataset, y=Target Attribute
    Output : Split data of train test
    
    We are removing dev , as dev can be better used for the models that includes DNN, CNN and RNN
    
    """
    X=x
    Y=y
    print('Splitting')
    train_x,test_x,train_y,test_y = train_test_split(X,Y,test_size=0.2,random_state=12)
    
    #train_x : Training data with attributes
    #train_y : Training data with ratings
    #test_x  : testing data with attributes
    #test_y  : testing data with rating 
    
    print("Final size of train/test :",train_x.size,"/",test_x.size)
    return train_x,train_y,test_x,test_y

In [6]:
def tfidf_vect(train_x,test_x):
    
    """
    Input : train and text attributes needed to be vectorized
    Output : Vectorized versions of the input
    
    """
    print('Vectorizing')
    print("test_x :",test_x.shape )
    print("train_x :",train_x.shape )

    tfidf_vectorizer = TfidfVectorizer(stop_words='english',max_df=0.7)   # a TFIDF vectorizer $min_df=0.03
    tfidf_train_x = tfidf_vectorizer.fit_transform(train_x)  #fitting the training data
    tfidf_test_x = tfidf_vectorizer.transform(test_x)  #fitting the testing data
    print("tfidf_train_x : ",tfidf_train_x.shape)
    print("tfidf_test_x : ",tfidf_test_x.shape)
    return tfidf_train_x,tfidf_test_x    #returning a vectorized train and test data


In [7]:
def hash_vect(train_x,test_x):  
    """
    Input : train and text attributes needed to be vectorized
    Output : Vectorized versions of the input
    
    """
    print('Vectorizing')

    hash_vectorizer = HashingVectorizer(stop_words='english')  #creating a hashing vectorizer
    hash_train_x = np.absolute(hash_vectorizer.fit_transform(train_x)) #fitting the training data
    hash_test_x = np.absolute(hash_vectorizer.transform(test_x)) #fitting the testing data
    return hash_train_x,hash_test_x  #returning a vectorized train and test data

In [8]:
from sklearn.model_selection import GridSearchCV

def navie_bayes(train_x,train_y,test_x,test_y,itera=1000):
   
    print('model building')
    NBG = MultinomialNB()
    if itera == 1000:
        alphas=np.arange(0.001,1,0.001)
    else:
        alphas=np.arange(0.01,1,0.01)
    print('begin grid')
    grid = GridSearchCV(estimator=NBG, param_grid=dict(alpha=alphas))

    grid.fit(train_x, train_y)
    
    #print(grid)
    # summarize the results of the grid search
    print("grid score",grid.best_score_)
    print("grid alpha",grid.best_estimator_.alpha)
    alpha=grid.best_estimator_.alpha
    
    NB = MultinomialNB(alpha = alpha )
    NB.fit(train_x,train_y)  
    predict_NB_train = NB.predict(train_x) 
    predict_NB_test = NB.predict(test_x) 
    score_train = metrics.accuracy_score(train_y, predict_NB_train) 
    score_test = metrics.accuracy_score(test_y, predict_NB_test)
      
    print("Alpha: {:.3f} test_Score: {:.5f} train_score: {:.5f}".format(alpha,score_test,score_train))# checking tha accuracy 
       
    #print("best accuracy with Alpha: {:.2f} Score: {:.5f}".format(alpha_value, best_score))
    cm_train= metrics.confusion_matrix(train_y, predict_NB_train)
    print("training confusion matrix")
    print(cm_train)
    cm_test = metrics.confusion_matrix(test_y, predict_NB_test)
    print("testing confusion matrix")
    print(cm_test)
    
    print("------------------------------------------")
